In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw
from Diffusion_Functions import frame_adjust, MSD_Deff_Calcs, prettify2, fillin2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
name = '100nm_RED_PEG_NPs_in_1_2p_agarose'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)
Tplot = '{}_Tplot'.format(name)
T2plot = '{}_T2plot'.format(name)

conversion = dict()

conversion[1] = (0.10, 9.91, 1)

frames = 446
frames_1 = np.linspace(0, frames, frames+1).astype(np.int64)
time = frames_1/conversion[1][1]

In [3]:
with open('pM1x_0.csv', "rb") as f_handle:
    interim = np.genfromtxt(f_handle, delimiter=",")

interim.shape

(446, 1928)

In [4]:
SM1x = dict()
SM1y = dict()
SM2xy = dict()

size = 16
local_n = 1928

for num in range(0, size):
    with open('pM1x_{}.csv'.format(num), "rb") as f_handle:
        interim = np.genfromtxt(f_handle, delimiter=",")
    for i in range(1, local_n+1):
        current = local_n*num + i
        SM1x[current] = interim[:, i-1]
    with open('pM1y_{}.csv'.format(num), "rb") as f_handle:
        interim1 = np.genfromtxt(f_handle, delimiter=",")
    for i in range(1, local_n+1):
        current = local_n*num + i
        SM1y[current] = interim1[:, i-1]
    with open('pM2xy_{}.csv'.format(num), "rb") as f_handle:
        interim2 = np.genfromtxt(f_handle, delimiter=",")
    for i in range(1, local_n+1):
        current = local_n*num + i
        SM2xy[current] = interim2[:, i-1]

In [5]:
total_check = local_n*size

arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])

st_arM1x = np.zeros(SM1x[1].shape[0])
st_arM1y = np.zeros(SM1x[1].shape[0])
st_arM2xy = np.zeros(SM1x[1].shape[0])

st_arD1x = np.zeros(SM1x[1].shape[0])
st_arD1y = np.zeros(SM1x[1].shape[0])
st_arD2xy = np.zeros(SM1x[1].shape[0])

tots = total_check
time2 = time

gM1x = dict()
gM1y = dict()
gM2xy = dict()

log_gM1x = dict()
log_gM1y = dict()
log_gM2xy = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])

st_geoM1x = np.zeros(SM1x[1].shape[0])
st_geoM1y = np.zeros(SM1x[1].shape[0])
st_geoM2xy = np.zeros(SM1x[1].shape[0])

geoD1x = np.zeros(SM1x[1].shape[0])
geoD1y = np.zeros(SM1x[1].shape[0])
geoD2xy = np.zeros(SM1x[1].shape[0])

st_geoD1x = np.zeros(SM1x[1].shape[0])
st_geoD1y = np.zeros(SM1x[1].shape[0])
st_geoD2xy = np.zeros(SM1x[1].shape[0])

for num2 in range(0, SM1x[1].shape[0]):
    gM1x[num2+1] = np.zeros(tots)
    gM1y[num2+1] = np.zeros(tots)
    gM2xy[num2+1] = np.zeros(tots)

    for num in range(1, tots+1):
        gM1x[num2+1][num-1] = SM1x[num][num2]
        gM1y[num2+1][num-1] = SM1y[num][num2]
        gM2xy[num2+1][num-1] = SM2xy[num][num2]

    gM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
    gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
    gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])

    gM1x[num2+1] = ma.masked_equal(gM1x[num2+1], 0)
    gM1y[num2+1] = ma.masked_equal(gM1y[num2+1], 0)
    gM2xy[num2+1] = ma.masked_equal(gM2xy[num2+1], 0)

    log_gM1x[num2+1] = np.log(gM1x[num2+1])
    log_gM1y[num2+1] = np.log(gM1y[num2+1])
    log_gM2xy[num2+1] = np.log(gM2xy[num2+1])

    geoM1x[num2] = stat.gmean(gM1x[num2+1])
    geoM1y[num2] = stat.gmean(gM1y[num2+1])
    geoM2xy[num2] = stat.gmean(gM2xy[num2+1])

    st_geoM1x[num2] = np.abs(geoM1x[num2]-np.exp(np.mean(np.log(gM1x[num2+1]))-np.std(np.log(gM1x[num2+1]))/np.sqrt(gM1x[num2+1].shape[0])))
    st_geoM1y[num2] = np.abs(geoM1y[num2]-np.exp(np.mean(np.log(gM1y[num2+1]))-np.std(np.log(gM1y[num2+1]))/np.sqrt(gM1y[num2+1].shape[0])))
    st_geoM2xy[num2] = np.abs(geoM2xy[num2]-np.exp(np.mean(np.log(gM2xy[num2+1]))-np.std(np.log(gM2xy[num2+1]))/np.sqrt(gM2xy[num2+1].shape[0])))

    geoD1x[num2] = geoM1x[num2]/time2[num2]
    geoD1y[num2] = geoM1y[num2]/time2[num2]
    geoD2xy[num2] = geoM2xy[num2]/time2[num2]

    arM1x[num2] = np.mean(gM1x[num2+1])
    arM1y[num2] = np.mean(gM1y[num2+1])
    arM2xy[num2] = np.mean(gM2xy[num2+1])

    st_arM1x[num2] = np.std(gM1x[num2+1])
    st_arM1y[num2] = np.std(gM1y[num2+1])
    st_arM2xy[num2] = np.std(gM2xy[num2+1])

    arD1x[num2] = arM1x[num2]/time2[num2]
    arD1y[num2] = arM1y[num2]/time2[num2]
    arD2xy[num2] = arM2xy[num2]/time2[num2]

    st_arD1x[num2] = st_arM1x[num2]/time2[num2]
    st_arD1y[num2] = st_arM1y[num2]/time2[num2]
    st_arD2xy[num2] = st_arM2xy[num2]/time2[num2]

np.savetxt('geoD2xy_{}.csv'.format(name), geoD2xy, delimiter=',')
np.savetxt('arD2xy_{}.csv'.format(name), arD2xy, delimiter=',')
#np.savetxt('FD2xy_{}.csv'.format(name), FD2xy, delimiter=',')

np.savetxt('geoM2xy_{}.csv'.format(name), geoM2xy, delimiter=',')
np.savetxt('arM2xy_{}.csv'.format(name), arM2xy, delimiter=',')
#np.savetxt('FM2xy_{}.csv'.format(name), FM2xy, delimiter=',')

np.savetxt('geoM1x_{}.csv'.format(name), geoM1x, delimiter=',')
np.savetxt('arM1x_{}.csv'.format(name), arM1x, delimiter=',')
np.savetxt('geoM1y_{}.csv'.format(name), geoM1y, delimiter=',')
np.savetxt('arM1y_{}.csv'.format(name), arM1y, delimiter=',')

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [6]:
def plot_histogram(M, time, total, datatype, filename, tn):
    """
    This function plots a histogram of the desired input, typically an MSD dataset or a diffusion dataset.
    
    Inputs: 
    M: MSD or Deff dataset
    time: time np array
    total: total number of particles
    datatype:
    filename: 
    tn: 
    """
    total1 = total
    MSD = M

    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]

    hist = [x for x in hist if str(x) != 'nan']
    
    plot, bins = np.histogram(hist, bins=40)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel(r'MSDs ($\mu$m$^2$) at $\tau$ = 1s', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [7]:
"""
Plots a histogram of calculated diffusion coefficients at 5 seconds.
"""

a, hist, total= plot_histogram(SM2xy, time2, tots, 'MSD (um^2)', Hplot, 1)
plt.show()

In [8]:
def plot_loghistogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.
    """

    total1 = total
    MSD = M
    
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]
    
    z1, z2 = find_nearest(hist[1:], 0)
    hist = np.delete(hist[1:], z2)
    hist = [x for x in hist if str(x) != 'nan']
    loghist = np.log(hist)

    plot, bins = np.histogram(loghist, bins=30)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel(r'logarithm of the MSDs ($\mu$m$^2$) at $\tau$ = 1s', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [9]:
"""
Plots a histogram of the logarithm of the diffusion coefficients at 1 second.
"""

hist, MSD, total = plot_loghistogram(SM2xy, time2, tots, 'MSD (um^2)', Hlogplot, 1)
plt.show()

In [10]:
tots

30848